In [2]:
import pandas as pd
import numpy as np

In [4]:
volume_cryptos = pd.read_pickle('./volume_cryptos.pkl')
volume_cryptos.head()

,"(11, 1)","(11, 2)","(11, 3)","(11, 4)","(11, 5)","(11, 6)","(11, 7)","(11, 8)","(11, 9)","(11, 10)",...,"(2, 20)","(2, 21)","(2, 22)","(2, 23)","(2, 24)","(2, 25)","(2, 26)","(2, 27)","(2, 28)","(2, 29)"
0,8.97077,9.52448,27.7077,28.2831,54.9851,78.7985,24.0819,33.2921,63.0435,6.37282,...,240.041,34.5393,16.8097,20.9787,146.763,243.769,346.125,132.473,163.899,72.0337
1,43.6276,213.665,61.8671,21.0084,21.4793,18.5754,77.7306,12.5094,72.7155,6.57368,...,186.461,138.815,82.3437,1.63956,132.843,29.1683,201.189,972.228,239.794,26.4632
2,11.6722,40.4598,33.2271,8.03392,10.453,11.5149,203.406,4.09453,2.22694,4.11494,...,421.362,86.2996,78.2372,5.75314,549.869,113.372,177.432,405.982,63.7563,35.4816
3,31.1001,5.30367,21.1073,1.75745,168.712,21.6921,13.9024,54.2846,1.26975,45.7626,...,435.134,35.3558,55.9405,256.768,204.554,37.7803,121.499,131.741,44.9759,26.2666
4,7.15688,3.05717,79.5344,7.33955,24.5172,5.23118,13.1275,12.9724,88.4375,11.0792,...,138.428,28.5055,4.95093,17.0059,176.265,31.8712,158.496,105.506,56.4955,291.459


In [5]:
volume_days = pd.read_pickle('./volume_days.pkl')
volume_days.head()

,BCH-USD,BTC-USD,ETH-USD,LTC-USD
0,53.5413,22.1059,167.07,241.191
1,44.7915,14.4794,105.412,177.137
2,32.0813,12.0744,108.982,166.337
3,31.7929,13.0342,95.402,127.053
4,25.0684,9.5097,73.9908,110.165
